In [ ]:
%load_ext autoreload
%autoreload 2

from functools import partial

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import seaborn as sns
import shap
import xgboost as xgb
from scipy.ndimage import gaussian_filter
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

from biobank_olink.constants import PROJECT_DATA

nb_data = PROJECT_DATA / "olink"
olink_parquet = nb_data / "Olink_whole_caucasian_nonrandom_Zscore.parquet"

TEST_SIZE = 0.2
MAX_SHAP_FEATS = 30

sns.set_theme()
pl.__version__

In [ ]:
import polars.selectors as cs

feat2code = {
    "SBP": "4080",
    "DBP": "4079",
    "ASI": "21021",
    "HR": "102",
    "DATE": "53",
    "DRUGS_Q1": "6177",
    "DRUGS_Q2": "6153",

}
instances = [0, 1, 2, 3]

df_followups = (
    pl.scan_csv(nb_data / "BP_ASI_pulserate_instances0123_participant.csv").join(
        pl.scan_csv(nb_data / "Drugs_instances123_participant.csv", infer_schema_length=1_000_000),
        on="eid"
    )
    .select(
        "eid",
        *[
            pl.concat_list(cs.contains(f"{feat2code[feat]}-{ins}.")).list.mean().alias(
                f"{feat}/{ins}")
            for feat in ["SBP", "DBP", "HR"]
            for ins in instances
        ],
        *[
            pl.col(f"{feat2code['ASI']}-{ins}.0").cast(pl.Float64).alias(f"{'ASI'}/{ins}")
            for ins in instances
        ],
        *[
            pl.col(f"{feat2code['DATE']}-{ins}.0").str.to_date("%d/%m/%Y").alias(f"{'DATE'}/{ins}")
            for ins in instances
        ],
        *[
            pl.any_horizontal(
                pl.col(f"{feat2code[feat]}-{ins}.0").str.contains("2")
                for feat in ["DRUGS_Q1", "DRUGS_Q2"]
            ).fill_null(False).alias(f"BP_DRUGS/{ins}")
            for ins in instances[1:]
        ],
    )
    .with_columns(
        (pl.col("^DATE/[123]$") - pl.col("DATE/0")).dt.total_days() / 365.25
    )
    .drop("DATE/0")
    .unpivot(index="eid")
    .filter(pl.col("value").is_not_null())
    .with_columns(
        pl.col("variable").str.split_exact("/", 1)
    )
    .with_columns(
        variable=pl.col("variable").struct[0],
        ins_index=pl.col("variable").struct[1].cast(pl.Int64)
    )
    .collect()
    .pivot(index=["eid", "ins_index"], on="variable")
    .rename({"DATE": "followup_time"})
    # Remove rows where a patient does take BP drugs,
    # so BP_DRUGS is null or 0 (falseo)
    .filter(pl.col("BP_DRUGS").is_null() | (pl.col("BP_DRUGS") == 0))
    .drop("BP_DRUGS")
    .with_columns(PP=pl.col("SBP") - pl.col("DBP"))
    .with_columns((pl.col("PP") * pl.col("HR")).alias("PP*HR"))
)

In [ ]:
def create_target_at_followups(target) -> pl.DataFrame:
    df_baseline = (
        df_followups
        .filter(pl.col("ins_index") == 0, pl.col(target).is_not_null())
        .rename({target: "target_at_baseline"})
        .select("eid", "PP", "PP*HR", "target_at_baseline")
    )
    return (
        df_followups
        .filter(
            pl.col("ins_index") != 0,
            pl.col(target).is_not_null(),
        )
        .select("eid", target, "followup_time")
        .join(df_baseline, on="eid")
        .select(
            "eid",
            "followup_time",
            "PP",
            "PP*HR",
            target=(pl.col(target) - pl.col("target_at_baseline")) / pl.col("followup_time")
        )
    )


In [ ]:
index_cols = ["eid", "sex", "age", "BMI", "Smokinstatus"]
cols = ["protein_id", "result"] + index_cols

coding = (
    pl.read_csv(nb_data / "coding143.tsv", separator="\t")
    .with_columns(pl.col("meaning").str.split(";").list.get(0))
)


def create_dataset(target) -> pl.DataFrame:
    df_target = create_target_at_followups(target)
    if target == "ASI":
        df_target = df_target.filter(pl.col("target") < 25)
    df_ol = (
        pl.scan_parquet(olink_parquet.with_suffix(".parquet"))
        .filter(
            pl.col("HTNgroup").is_in([0, 1]),
            pl.col("eid").is_in(df_target["eid"]),
        )
        .select(cols)
        .with_columns(
            pl.col("protein_id").replace_strict(
                coding["coding"], coding["meaning"], return_dtype=pl.String
            )
        )
        .collect()
        .pivot(on="protein_id", index=index_cols, values="result",
               aggregate_function="first")
    )
    df_ol = (
        df_ol.with_columns(
            df_ol["Smokinstatus"].to_dummies(drop_first=True)
        )
        .drop("Smokinstatus")
    )
    return df_ol.join(df_target, on="eid", how="inner").drop("eid")

In [ ]:
import wandb


def shap_to_importance(values, feat_names) -> pd.Series:
    return (
        pd.Series(np.abs(values.values).sum(axis=0), index=feat_names)
        .sort_values(ascending=False)
    )


def print_true_vs_pred(y_true, y_pred, title="Train"):
    ax = sns.scatterplot(x=y_true, y=y_pred, alpha=0.4)
    max_lim = max(ax.get_xlim()[1], ax.get_ylim()[1])
    min_lim = min(ax.get_xlim()[0], ax.get_ylim()[0])
    ax.set(xlabel="y_true", ylabel="y_pred", title=title,
           xlim=(min_lim, max_lim), ylim=(min_lim, max_lim))


def setup_xgb(config, seed=42, device="gpu", is_regression: bool = True) -> xgb.XGBRegressor:
    xgb_params = dict(config)

    early_stop = xgb.callback.EarlyStopping(
        rounds=10, save_best=True
    )

    model = xgb.XGBRegressor if is_regression else xgb.XGBClassifier
    return model(
        tree_method="hist",
        device=device,
        random_state=seed,
        callbacks=[early_stop],
        **xgb_params,
    )


def eval_xgb(x, y, config, seed=42, device="gpu", is_regression: bool = True) -> xgb.XGBRegressor:
    if not is_regression:
        y = y > 0
        print(f"Prevalence={y.mean():.2%}")
        
    x_train, x_eval, y_train, y_eval = train_test_split(x, y, test_size=TEST_SIZE,
                                                        random_state=seed)

    clf = setup_xgb(config, seed, device, is_regression)
    clf.fit(x_train, y_train, eval_set=[(x_eval, y_eval)], verbose=10)

    y_train_pred = clf.predict(x_train)
    if is_regression:
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 2, 1)
        corr_train = np.corrcoef(y_train, y_train_pred)[1, 0]
        print_true_vs_pred(y_train, y_train_pred,
                           title=f"Train (r2={clf.score(x_train, y_train):.4f}, corr={corr_train:.4f})")
    else:
        f1 = f1_score(y_train, y_train_pred)
        print(f"Train (acc={clf.score(x_train, y_train):.4f}, f1={f1:.4f})")
        
    y_eval_pred = clf.predict(x_eval)
    if is_regression:
        plt.subplot(1, 2, 2)
        corr_eval = np.corrcoef(y_eval, y_eval_pred)[1, 0]
        print_true_vs_pred(y_eval, y_eval_pred,
                           title=f"Eval (r2={clf.score(x_eval, y_eval):.4f}, corr={corr_eval:.4f})")
    else:
        f1 = f1_score(y_eval, y_eval_pred)
        print(f"Eval (acc={clf.score(x_eval, y_eval):.4f}, f1={f1:.4f})")
               
    return clf


def get_elbow_point(series: pd.Series) -> int:
    from kneed import KneeLocator
    
    y = series.values
    y = y[y != 0]
    y_fit = gaussian_filter(y, sigma=5)
    x = np.arange(0, len(y))
    
    kneedle = KneeLocator(x, y_fit, curve='convex', direction='decreasing')
    elbow_point = kneedle.elbow

    ax = series.plot.line()
    ax.axvline(x=elbow_point, color='k', linestyle='--', lw=1)
    plt.plot(x, y_fit, label='Smoothened Data')
    return elbow_point


def get_xy(target) -> tuple[pl.DataFrame, pl.Series]:
    df = create_dataset(target)
    x, y = df.drop("target"), df["target"]
    print("X shape:", x.shape)
    return x, y

## SDB

In [ ]:
x_sbp, y_sbp = get_xy("SBP")
best_run  = wandb.Api().sweep("olink_change_sbp/j22tre2d").best_run(order="r2")
model_sbp = eval_xgb(x_sbp, y_sbp, best_run.config)

In [ ]:
explainer = shap.TreeExplainer(model_sbp)
shap_values_sbp = explainer(x_sbp)
imps = shap_to_importance(shap_values_sbp, x_sbp.columns)
sbp_important_feats = imps[:get_elbow_point(imps)]
sbp_important_feats.index.values, f"Number of important feats: {len(sbp_important_feats)}"

In [ ]:
shap.summary_plot(shap_values_sbp, x_sbp.to_pandas(),
                  max_display=min(len(sbp_important_feats), MAX_SHAP_FEATS))

### SBP sign of the change classification

In [ ]:
x, y = get_xy("SBP")
best_run = wandb.Api().sweep("olink_change_sbp_clf/kurqw000").best_run(order="acc")
eval_xgb(x, y, best_run.config, is_regression=False)
""

## DBP

In [ ]:
x_dbp, y_dbp = get_xy("DBP")
best_run = wandb.Api().sweep("olink_change_dbp/594pl5su").best_run(order="r2")
model_dbp = eval_xgb(x_dbp, y_dbp, best_run.config)

In [ ]:
explainer = shap.TreeExplainer(model_dbp)
shap_values_dbp = explainer(x_dbp)
imps = shap_to_importance(shap_values_dbp, x_dbp.columns)
dbp_important_feats = imps[:get_elbow_point(imps)]
dbp_important_feats.index.values, f"Number of important feats: {len(dbp_important_feats)}"

In [ ]:
shap.summary_plot(shap_values_dbp, x_dbp.to_pandas(),
                  max_display=min(len(dbp_important_feats), MAX_SHAP_FEATS))

## ASI

In [ ]:
x_asi, y_asi = get_xy("ASI")
best_run = wandb.Api().sweep("olink_change_asi/v8hop5pw").best_run(order="r2")
model_asi = eval_xgb(x_asi, y_asi, best_run.config)

In [ ]:
explainer = shap.TreeExplainer(model_asi)
shap_values_asi = explainer(x_asi)
imps = shap_to_importance(shap_values_asi, x_asi.columns)
asi_important_feats = imps[:get_elbow_point(imps)]
asi_important_feats.index.values, f"Number of important feats: {len(asi_important_feats)}"

In [ ]:
shap.summary_plot(shap_values_asi, x_asi.to_pandas(),
                  max_display=min(len(asi_important_feats), MAX_SHAP_FEATS))

### ASI sign of the change classification

In [ ]:
x_asi, y_asi = get_xy("ASI")
best_run = wandb.Api().sweep("olink_change_asi_clf/2sb10t8z").best_run(order="acc")
model_asi = eval_xgb(x_asi, y_asi, best_run.config, is_regression=False)

In [ ]:
explainer = shap.TreeExplainer(model_asi)
shap_values_asi = explainer(x_asi)
imps = shap_to_importance(shap_values_asi, x_asi.columns)
asi_important_feats = imps[:get_elbow_point(imps)]
asi_important_feats.index.values, f"Number of important feats: {len(asi_important_feats)}"

In [ ]:
shap.summary_plot(shap_values_asi, x_asi.to_pandas(),
                  max_display=min(len(asi_important_feats), MAX_SHAP_FEATS))

## Wandb sweep

In [ ]:
def wandb_eval_xgb(config, x, y, seed=42, is_regression: bool = True):
    if not is_regression:
        y = y > 0
    
    X_train, X_eval, y_train, y_eval = train_test_split(x, y, test_size=TEST_SIZE,
                                                        random_state=seed)

    estimator = setup_xgb(config, seed, is_regression=is_regression)
    estimator.fit(X_train, y_train, eval_set=[(X_eval, y_eval)], verbose=0)
    return estimator.score(X_eval, y_eval)


is_reg = False
target = "ASI"
project_name = f"olink_change_{target.lower()}" + ("" if is_reg else "_clf")
x, y = get_xy(target)
objective = partial(wandb_eval_xgb, x=x, y=y, is_regression=is_reg)


def main():
    wandb.init(project=project_name)
    score = max(objective(wandb.config), -10)
    wandb.log({"r2" if is_reg else "acc": score})


sweep_configurations = {
    "method": "bayes",
    "metric": {"name": "r2" if is_reg else "acc", "goal": "maximize"},
    "parameters": {
        'n_estimators': {'distribution': 'int_uniform', 'min': 50, 'max': 500},
        'max_depth': {'distribution': 'int_uniform', 'min': 1, 'max': 10},
        'learning_rate': {'distribution': 'log_uniform_values', 'min': 0.001, 'max': 0.1},
        'subsample': {'distribution': 'uniform', 'min': 0.5, 'max': 1.0},
        'colsample_bytree': {'distribution': 'uniform', 'min': 0.5, 'max': 1.0},
        'gamma': {'distribution': 'uniform', 'min': 0, 'max': 10},
        'min_child_weight': {'distribution': 'int_uniform', 'min': 1, 'max': 20},
        'reg_alpha': {'distribution': 'log_uniform_values', 'min': 1e-5, 'max': 100},
        'reg_lambda': {'distribution': 'log_uniform_values', 'min': 1e-5, 'max': 100},
    },
}

#sweep_id = wandb.sweep(sweep_configurations, project=project_name)
#sweep_id

In [ ]:
#wandb.teardown()
#wandb.agent(sweep_id, function=main, count=200)